In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Initialize the WebDriver
driver = webdriver.Chrome()  # You can replace Chrome with Firefox or any other browser

# Open the URL
url = "https://obd.hcraontario.ca/buildersearchresults?builderLocation=toronto&page=1"
driver.get(url)

# Apply the filter (Adjust the selector based on the actual filter element)
filter_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "filterBtn"))
)
filter_button.click()

# Wait for the filter modal to appear
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "Modal_modal__3ulyu"))
)

# Select the filters
licensed_radio_button = driver.find_element(By.ID, "licensedBool_yes")
licensed_radio_button.click()

city_checkbox = driver.find_element(By.ID, "city_toronto_0")
city_checkbox.click()

# Wait for 5 seconds before closing the modal
time.sleep(5)

# Close the modal
close_modal_button = driver.find_element(By.CLASS_NAME, "Modal_closeBtn__BirQ7")
close_modal_button.click()

# Function to scrape links from the current page
def scrape_links_from_page():
    links = []
    rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
    for row in rows:
        link_element = row.find_element(By.CSS_SELECTOR, "a.title")
        link = link_element.get_attribute("href")
        links.append(link)
    return links

# Initialize an empty list to store all links
all_links = []

# Loop through all pages and scrape links
while True:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "tbody tr"))
    )
    links = scrape_links_from_page()
    all_links.extend(links)
    print(f"Scraped links from current page")

    # Try to find the "Next" button
    try:
        next_button = driver.find_element(By.XPATH, "//button/span[text()='Next']/parent::button")
        next_button.click()
        # Wait for the next page to load
        time.sleep(5)
    except:
        print("No more pages to scrape.")
        break

# Close the WebDriver
driver.quit()

# Save the links to a CSV file
df_links = pd.DataFrame(all_links, columns=['Profile Link'])
csv_file_path = "Builder_Profile_Links.csv"
df_links.to_csv(csv_file_path, index=False)

print(f"Data saved to {csv_file_path}")


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Load the CSV file containing the profile links
csv_file_path = "Builder_Profile_Links.csv"
profile_links_df = pd.read_csv(csv_file_path)

# Initialize the WebDriver
driver = webdriver.Chrome()  # You can replace Chrome with Firefox or any other browser

# Function to scrape content from a profile page
def scrape_profile_content(url):
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
    )
    time.sleep(5)  # Ensure the page loads completely
    soup = BeautifulSoup(driver.page_source, "html.parser")

    data = {
        "URL": url,
        "Vendor/Builder Legal Name": "N/A",
        "Doing Business As Name": "N/A",
        "Address": "N/A",
        "Website": "N/A",
        "Email": "N/A",
        "Phone Number": "N/A",
        "Fax Number": "N/A",
    }

    # Extract Vendor/Builder Legal Name and Doing Business As Name
    name_container = soup.find("div", class_="ProfileHeader_nameContainer__CQ7w-")
    if name_container:
        legal_name = name_container.find("h1", class_="ProfileHeader_name__22ueK")
        dba_name = name_container.find("h2", class_="ProfileHeader_name__22ueK")
        if legal_name:
            data["Vendor/Builder Legal Name"] = legal_name.get_text(strip=True)
        if dba_name:
            data["Doing Business As Name"] = dba_name.get_text(strip=True)

    overview_section = soup.find("div", {"id": "overview-tab"})
    if overview_section:
        fields = overview_section.find_all("div", style="box-sizing: border-box; min-height: 1px; position: relative; padding-left: 15px; padding-right: 15px; width: 50%; flex: 0 0 50%; max-width: 50%; margin-left: 0%; right: auto; left: auto;")
        for field in fields:
            key_element = field.find("span", class_="bold")
            value_element = field.find("p")
            if key_element and value_element:
                key = key_element.get_text(strip=True)
                value = value_element.get_text(strip=True)
                # Normalize keys to match expected output
                if key == "Phone Number":
                    key = "Phone Number"
                elif key == "Fax Number":
                    key = "Fax Number"
                elif key == "Email":
                    key = "Email"
                elif key == "Website":
                    key = "Website"
                elif key == "Address":
                    key = "Address"
                data[key] = value

    # Extracting principals, directors, and officers
    principals_section = soup.find("div", class_="overflowContainer")
    if principals_section:
        table = principals_section.find("table")
        if table:
            rows = table.find_all("tr")
            for i, row in enumerate(rows[1:], start=1):  # Skipping the header row
                cols = row.find_all("td")
                if len(cols) >= 3:
                    data[f"Principal/Director/Officer {i} Name"] = cols[0].get_text(strip=True)
                    data[f"Principal/Director/Officer {i} Role(s)"] = cols[1].get_text(strip=True)
                    data[f"Principal/Director/Officer {i} Contact"] = cols[2].get_text(strip=True)

    return data

# Initialize an empty list to store all profile data
all_profiles = []

# Loop through each profile URL in the CSV file and scrape content
for url in profile_links_df['Profile Link']:
    profile_data = scrape_profile_content(url)
    all_profiles.append(profile_data)
    print(f"Scraped data from {url}")

# Close the WebDriver
driver.quit()

# Save the scraped data to an Excel file
df_profiles = pd.DataFrame(all_profiles)
excel_file_path = "Profile_Data.xlsx"
df_profiles.to_excel(excel_file_path, index=False)

print(f"Data saved to {excel_file_path}")


In [ ]:
import pandas as pd

# Load the Excel file
file_path = "Profile_Data.xlsx"
df = pd.read_excel(file_path)

# Convert email column to lowercase
df['Email'] = df['Email'].str.lower()

# Define the conditions
na_emails = df[df['Email'].isna()]
regular_email_domains = r'@(gmail\.com|yahoo\.com|hotmail\.com|aol\.com|outlook\.com|icloud\.com|mail\.com|zoho\.com|protonmail\.com|gmx\.com|yandex\.com|live\.com|me\.com|mac\.com|msn\.com)'
regular_emails = df[df['Email'].str.contains(regular_email_domains, na=False, case=False)]
company_emails = df[~df['Email'].str.contains(regular_email_domains, na=False, case=False) & df['Email'].notna()]

# Split company emails into general and specific company emails
general_company_domains = r'^(info@|general@|contact@|admin@|sales@|support@)'
general_company_emails = company_emails[company_emails['Email'].str.contains(general_company_domains, na=False, case=False)]
specific_company_emails = company_emails[~company_emails['Email'].str.contains(general_company_domains, na=False, case=False)]

# Create a new Excel file with four sheets
with pd.ExcelWriter("Profile_Data_Modified.xlsx") as writer:
    na_emails.to_excel(writer, sheet_name='NA Emails', index=False)
    regular_emails.to_excel(writer, sheet_name='Regular Emails', index=False)
    general_company_emails.to_excel(writer, sheet_name='General Company Emails', index=False)
    specific_company_emails.to_excel(writer, sheet_name='Specific Company Emails', index=False)

print("Data has been successfully split and saved into Profile_Data_Modified.xlsx")


In [ ]:
import pandas as pd

# Load the Excel file
file_path = "Profile_Data_Modified.xlsx"
excel_file = pd.ExcelFile(file_path)

# Create a new Excel writer object
output_file_path = "Profile_Data_Transformed.xlsx"
writer = pd.ExcelWriter(output_file_path, engine='xlsxwriter')

# Function to pivot the data
def pivot_data(df):
    transformed_data = pd.DataFrame(columns=[
        'URL', 'Vendor/Builder Legal Name', 'Doing Business As Name', 'Address', 
        'Website', 'Email', 'Phone Number', 'Fax Number', 
        'Principal/Director/Officer Name', 'Role(s)', 'Contact'
    ])

    rows_to_add = []
    for index, row in df.iterrows():
        has_principals = False
        for i in range(1, 18):  # Adjust range to handle up to 7 entries
            name_col = f'Principal/Director/Officer {i} Name'
            role_col = f'Principal/Director/Officer {i} Role(s)'
            contact_col = f'Principal/Director/Officer {i} Contact'
            
            if pd.notna(row[name_col]):
                has_principals = True
                new_row = {
                    'URL': row['URL'],
                    'Vendor/Builder Legal Name': row['Vendor/Builder Legal Name'],
                    'Doing Business As Name': row['Doing Business As Name'],
                    'Address': row['Address'],
                    'Website': row['Website'],
                    'Email': row['Email'],
                    'Phone Number': row['Phone Number'],
                    'Fax Number': row['Fax Number'],
                    'Principal/Director/Officer Name': row[name_col],
                    'Role(s)': row[role_col],
                    'Contact': row[contact_col]
                }
                rows_to_add.append(new_row)
        
        # If no principals/officers were found, add a row with N/A
        if not has_principals:
            new_row = {
                'URL': row['URL'],
                'Vendor/Builder Legal Name': row['Vendor/Builder Legal Name'],
                'Doing Business As Name': row['Doing Business As Name'],
                'Address': row['Address'],
                'Website': row['Website'],
                'Email': row['Email'],
                'Phone Number': row['Phone Number'],
                'Fax Number': row['Fax Number'],
                'Principal/Director/Officer Name': 'N/A',
                'Role(s)': 'N/A',
                'Contact': 'N/A'
            }
            rows_to_add.append(new_row)
    
    transformed_data = pd.concat([transformed_data, pd.DataFrame(rows_to_add)], ignore_index=True)
    return transformed_data

# Process each sheet in the Excel file
for sheet_name in excel_file.sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    # Pivot the data
    transformed_data = pivot_data(df)
    # Write the transformed data to a new sheet in the output file
    transformed_data.to_excel(writer, sheet_name=sheet_name, index=False)

# Save the transformed data to the new Excel file
writer.close()

print("Data has been successfully transformed and saved into Profile_Data_Transformed.xlsx")


In [7]:
import pandas as pd

# Load the Excel file
file_path = "Profile_Data_Modified.xlsx"
excel_file = pd.ExcelFile(file_path)

# Create a new Excel writer object
output_file_path = "Profile_Data_Transformed.xlsx"
writer = pd.ExcelWriter(output_file_path, engine='xlsxwriter')

# Function to pivot the data
def pivot_data(df):
    transformed_data = pd.DataFrame(columns=[
        'URL', 'Vendor/Builder Legal Name', 'Doing Business As Name', 'Address', 
        'Website', 'Email', 'Phone Number', 'Fax Number', 
        'Principal/Director/Officer Name', 'Role(s)', 'Contact'
    ])
    
    for index, row in df.iterrows():
        has_principals = False
        for i in range(1, 27):  # Adjusted to 8 for up to 7 principals/directors/officers
            name_col = f'Principal/Director/Officer {i} Name'
            role_col = f'Principal/Director/Officer {i} Role(s)'
            contact_col = f'Principal/Director/Officer {i} Contact'
            
            if pd.notna(row[name_col]):
                has_principals = True
                new_row = {
                    'URL': row['URL'],
                    'Vendor/Builder Legal Name': row['Vendor/Builder Legal Name'],
                    'Doing Business As Name': row['Doing Business As Name'],
                    'Address': row['Address'],
                    'Website': row['Website'],
                    'Email': row['Email'],
                    'Phone Number': row['Phone Number'],
                    'Fax Number': row['Fax Number'],
                    'Principal/Director/Officer Name': row[name_col],
                    'Role(s)': row[role_col],
                    'Contact': row[contact_col]
                }
                transformed_data = pd.concat([transformed_data, pd.DataFrame([new_row])], ignore_index=True)
        
        # If no principals/officers were found, add a row with N/A
        if not has_principals:
            new_row = {
                'URL': row['URL'],
                'Vendor/Builder Legal Name': row['Vendor/Builder Legal Name'],
                'Doing Business As Name': row['Doing Business As Name'],
                'Address': row['Address'],
                'Website': row['Website'],
                'Email': row['Email'],
                'Phone Number': row['Phone Number'],
                'Fax Number': row['Fax Number'],
                'Principal/Director/Officer Name': 'N/A',
                'Role(s)': 'N/A',
                'Contact': 'N/A'
            }
            transformed_data = pd.concat([transformed_data, pd.DataFrame([new_row])], ignore_index=True)
    
    return transformed_data

# Process each sheet in the Excel file
for sheet_name in excel_file.sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    # Convert all values in the 'Email' column to strings and lowercase
    df['Email'] = df['Email'].astype(str).str.lower()
    # Sort the dataframe based on 'Vendor/Builder Legal Name'
    df = df.sort_values(by='Vendor/Builder Legal Name')
    # Pivot the data
    transformed_data = pivot_data(df)
    # Write the transformed data to a new sheet in the output file
    transformed_data.to_excel(writer, sheet_name=sheet_name, index=False)

# Save the transformed data to the new Excel file
writer.close()

print("Data has been successfully transformed and saved into Profile_Data_Transformed.xlsx")


Data has been successfully transformed and saved into Profile_Data_Transformed.xlsx
